# Threat Assessment Strategies

In [6]:
import math
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.path as path

from scipy.spatial import Voronoi

from shapely.geometry import LineString, Polygon, MultiPolygon, Point
from shapely.ops import polygonize, unary_union

import import_ipynb
import node_linker
import pps
import utils

## BVC Assembler

`def bvc_assembler(agent, mode, node, radius, verbose, vor_prox)`

Assemble the vertices for each cell in order to generate the new BVC vertices

### Parameters:
* **agent&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: class machines.Robot**
    * Current agent.
* **node&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: class node_linker.Node**
    * A list of ordered vertices for each robot.
* **mode&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: string**
    * "closed" or "open" according to the cell.
* **radius&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: float**
    * Safety radius to define the BVC from the Voronoi Diagram
* **verbose&nbsp;&nbsp;: boolean**
    * Flag for verbose
* **vor_prox : dictionary**
    * Approximated values for Voronoi generation

In [ ]:
def bvc_assembler(agent, mode, node, radius, verbose, vor_prox):
    # Auxiliar node for possible cuts
    aux_node = node.nref
    # Get coordinates
    vertex_start = vor_prox["vrtx"][node.item]
    vertex_mid1 = vor_prox["vrtx"][node.nref.item]
    vertex_mid2 = vor_prox["vrtx"][aux_node.item]
    vertex_end = vor_prox["vrtx"][aux_node.nref.item]
    # Get the new vertices in this sector
    sector = utils.intersect(vertex_mid1, radius, vertex_start, vertex_end)
    if mode == "open":
        # Assign the new values according special cases 
        if node is agent.cell.start_node:
            agent.bvc.append(sector[0])
        agent.bvc.append(sector[1])
        if aux_node.nref.nref is None:
            agent.bvc.append(sector[2])
    elif mode == "closed":
        if verbose:
            print(sector[1])
        agent.bvc.append(sector[1])

## BVC Generator

`def bvc_gen(radius, robots, verbose, vor_prox)`

Main function for the generation of the BVC for each agent

### Parameters:
* **radius&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: float**
    * Safety radius to define the BVC from the Voronoi Diagram
* **robots&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;list**
    * List of instances of Robots
* **verbose&nbsp;&nbsp;: boolean**
    * Flag for verbose
* **vor_prox : dictionary**
    * Approximated values for Voronoi generation

In [2]:
def bvc_gen(radius, robots, verbose, vor_prox):
    location = []
    set_of_new_vertices = [] # For storing the BVC vertices of each robot
    if verbose:
        print("==================================================")
        print("                 BVC GENERATION")
        print("==================================================")
    # Calculate the bvc vertices and the closest point
    for agent in robots:
        agent.bvc = []
        if verbose:
            print('Analyzing these vertices...')
            agent.cell.traverse_list()
        node = agent.cell.start_node
        # Go in pairs of edges
        # Check if the cell is unlimited
        if agent.cell.end_node.nref is None:
            if verbose:
                print('\nOPEN CELL!')
                print('Number of vertices in this cell:',
                      agent.cell.lenght_list,'\n')
            while node.nref.nref is not agent.cell.end_node.nref:
                bvc_assembler(agent, "open", node, radius, verbose, vor_prox)
                # Move the node 
                node = node.nref
            # Remove intersections of the BVC
            utils.pruner(agent, verbose)
        # If the cell is limited or closed
        else:
            if verbose:
                print('\nCLOSED CELL!')
                print('Number of vertices in this cell:',
                      agent.cell.lenght_list)
            for _ in range(agent.cell.lenght_list):
                bvc_assembler(agent, "closed", node, radius, verbose, vor_prox)
                # Move the node 
                node = node.nref
            # Remove intersections of the BVC
            ls = LineString(agent.bvc)
            lr = LineString(ls.coords[:] + ls.coords[0:1])
            if not lr.is_simple:
                if verbose:
                    print("\nThere is an intersection!")
                mls = unary_union(lr)
                for polygon in polygonize(mls):
                    if polygon.contains(Point(agent.pos_i)):
                        break;
                agent.bvc = [np.array(k) for k in polygon.exterior.coords]
            else:
                agent.bvc.append(agent.bvc[0])
        if verbose:
            print('The new vertices are:',type(agent.bvc[0]))
            for vertex in agent.bvc:
                print(vertex)
            print('---------------------------------------------------------')
        # Check if target point is in/out of BVC 
        agent.inside = path.Path(np.array(agent.bvc)).contains_point(agent.pos_f)

## Voronoi Diagram Generator

`def vor_prox(pos_i, verbose, vor)`

Generate the main Voronoi Diagram approximating the values of infinity vertices  
and returning the updated values of edges and vertices for BVC constructor.

### Parameters:
* **pos_i&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: list**
    * List of current positions of all agents
* **verbose : bool**
    * Flag for verbose
* **vor&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;:
class scipy.spatial.Voronoi**
    * Voronoi diagram attributes
   
### Return:
* **edges&nbsp;&nbsp;&nbsp;&nbsp;: ndarray of int**
    * Updated pointers to vertices for each edge
* **vertices : ndarray of float**
    * Updated vertices with approximation of infinity vertices

In [ ]:
def vor_prox(pos_i, verbose, vor):
    count = 0
    # Create copies of:
    vertices = vor.vertices
    edges = []
    # Find the center of the robots position
    center = pos_i.mean(axis=0)
    # pointidx: Indices of the points between which each Voronoi ridge lies.
    # simplex: Indices of the Voronoi vertices forming each Voronoi ridge.
    # zip join each values in a tuple
    for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
        # Transform to array
        simplex_arr = np.array(simplex)
        # If the Voronoi edge goes to infinity
        if np.all(simplex_arr >= 0):
            # Append to the BVC edges list
            edges.append(simplex)
        # If one of the vertices goes to the infinity
        else:
            # Get the finite vertex
            i = simplex_arr[simplex_arr >= 0][0] 
            # Calculate the straight line between robots
            t = pos_i[pointidx[1]] - pos_i[pointidx[0]] 
            # Normalization of this line
            t = t / np.linalg.norm(t)
            # Ortogonalization
            n = np.array([-t[1], t[0]])
            # Get the midpoint between robots
            midpoint = pos_i[pointidx].mean(axis=0)
            # Far point = extension from the finite vertex using the
            #             orthogonalization and
            #             depending of the midpoint location respect center
            far_point = vor.vertices[i] + (np.sign(np.dot(midpoint- center,n))
                                           * n * 20 * len(pos_i))
            # Increase the count
            count += 1
            # Add the value to the BVC vertices array
            vertices = np.concatenate((vertices, far_point.reshape(1,2)),
                                      axis=0)
            # Update the pointer of BVC edges list 
            edges.append([len(vertices)-1, simplex[1]])
    if verbose:
        print('List of vertex coordinates (including approximations) of',
              'Voronoi Diagram:\n')
        for i in range(len(vertices)):
            print(i,".-",vertices[i])
        print('Robots in front: ',vor.ridge_points,'\n')
        print('Voronoi Diagram Edges: ',edges,'\n')
    return {"vrtx" : vertices, "edge" : edges}